# Extracting Data from here.com
<br>

Author: Bala Krishnamoorthy

#### Goal

Access here.com's api to extract information on road closures, incidents, etc.

### Data Gathering

In [11]:
# Import Libraries

import json, requests
import time, datetime
import pandas as pd

# Viewing configs for notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'facecolor' : 'w'}
# Code below allows console outputs to be generated without print statements
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

In [12]:
# Imports my API credentials stored locally. These are **not** publically available. To use
# the API access code below, please create your own access credentials on here.com and execute
# the code below. 
import config 

In [17]:
# City Coordinates

# Boston
top_left = '42.366,-71.066;'
bot_right = '42.329,-71.033'

In [18]:
url = 'https://traffic.api.here.com/traffic/6.3/\
incidents.json?app_id=' + config.app_id + '&app_code=' + config.app_code + '&bbox=' + \
top_left + bot_right
url;

In [19]:
res = requests.get(url)

In [20]:
res.status_code

200

In [22]:
hmaps_json = res.json()
hmaps_json;

In [23]:
# Inspect json file
hmaps_json.keys()
hmaps_json['TRAFFIC_ITEMS'].keys()
hmaps_json['diagnostic'].keys()
hmaps_json['TRAFFIC_ITEMS']['TRAFFIC_ITEM'];

In [52]:
incident_dict = hmaps_json['TRAFFIC_ITEMS']['TRAFFIC_ITEM']
incident_dict[0].keys()

dict_keys(['TRAFFIC_ITEM_ID', 'ORIGINAL_TRAFFIC_ITEM_ID', 'TRAFFIC_ITEM_STATUS_SHORT_DESC', 'TRAFFIC_ITEM_TYPE_DESC', 'START_TIME', 'END_TIME', 'ENTRY_TIME', 'CRITICALITY', 'VERIFIED', 'ABBREVIATION', 'LOCATION', 'TRAFFIC_ITEM_DETAIL', 'TRAFFIC_ITEM_DESCRIPTION', 'mid', 'PRODUCT'])

In [53]:
incident_df = pd.DataFrame(incident_dict)
incident_df.head()
incident_df.shape

,ABBREVIATION,CRITICALITY,END_TIME,ENTRY_TIME,LOCATION,ORIGINAL_TRAFFIC_ITEM_ID,PRODUCT,START_TIME,TRAFFIC_ITEM_DESCRIPTION,TRAFFIC_ITEM_DETAIL,TRAFFIC_ITEM_ID,TRAFFIC_ITEM_STATUS_SHORT_DESC,TRAFFIC_ITEM_TYPE_DESC,VERIFIED,mid
0,"{'SHORT_DESC': 'CLOS', 'DESCRIPTION': ''}","{'ID': '0', 'DESCRIPTION': 'critical'}",01/12/2019 02:38:51,01/11/2019 21:40:54,"{'INTERSECTION': {'ORIGIN': {'ID': '', 'STREET...",3018985707296936927,basic,01/11/2019 21:40:54,[{'value': 'Closed between Causeway St and Ant...,"{'ROAD_CLOSED': True, 'INCIDENT': {'RESPONSE_V...",4569300205414887358,ACTIVE,ROAD_CLOSURE,True,NAVTEQ/r_NAVTEQ/11090482_TIC/NAVTEQ|1547242848562
1,"{'SHORT_DESC': 'CONST', 'DESCRIPTION': 'constr...","{'ID': '2', 'DESCRIPTION': 'minor'}",01/15/2019 09:38:51,01/11/2019 09:39:20,"{'INTERSECTION': {'ORIGIN': {'ID': '', 'STREET...",1037353930832550411,basic,01/11/2019 09:39:20,[{'value': 'Between E St and F St - Constructi...,"{'ROAD_CLOSED': False, 'EVENT': {'EVENT_ITEM_C...",1037353930832550411,ACTIVE,CONSTRUCTION,True,NAVTEQ/r_NAVTEQ/11431707_TIC/NAVTEQ|1547199531854


(2, 15)

In [ ]:
# Only two entries in specified bounding box

In [56]:
incident_df_2 = incident_df.drop(axis=1, columns=['ABBREVIATION', 'ORIGINAL_TRAFFIC_ITEM_ID', 'PRODUCT', 
                                  'TRAFFIC_ITEM_ID', 'mid'])
incident_df_2

,CRITICALITY,END_TIME,ENTRY_TIME,LOCATION,START_TIME,TRAFFIC_ITEM_DESCRIPTION,TRAFFIC_ITEM_DETAIL,TRAFFIC_ITEM_STATUS_SHORT_DESC,TRAFFIC_ITEM_TYPE_DESC,VERIFIED
0,"{'ID': '0', 'DESCRIPTION': 'critical'}",01/12/2019 02:38:51,01/11/2019 21:40:54,"{'INTERSECTION': {'ORIGIN': {'ID': '', 'STREET...",01/11/2019 21:40:54,[{'value': 'Closed between Causeway St and Ant...,"{'ROAD_CLOSED': True, 'INCIDENT': {'RESPONSE_V...",ACTIVE,ROAD_CLOSURE,True
1,"{'ID': '2', 'DESCRIPTION': 'minor'}",01/15/2019 09:38:51,01/11/2019 09:39:20,"{'INTERSECTION': {'ORIGIN': {'ID': '', 'STREET...",01/11/2019 09:39:20,[{'value': 'Between E St and F St - Constructi...,"{'ROAD_CLOSED': False, 'EVENT': {'EVENT_ITEM_C...",ACTIVE,CONSTRUCTION,True


In [61]:
incident_df_2.rename(str.lower, axis=1, inplace=True)
incident_df_2

,criticality,end_time,entry_time,location,start_time,traffic_item_description,traffic_item_detail,traffic_item_status_short_desc,traffic_item_type_desc,verified
0,"{'ID': '0', 'DESCRIPTION': 'critical'}",01/12/2019 02:38:51,01/11/2019 21:40:54,"{'INTERSECTION': {'ORIGIN': {'ID': '', 'STREET...",01/11/2019 21:40:54,[{'value': 'Closed between Causeway St and Ant...,"{'ROAD_CLOSED': True, 'INCIDENT': {'RESPONSE_V...",ACTIVE,ROAD_CLOSURE,True
1,"{'ID': '2', 'DESCRIPTION': 'minor'}",01/15/2019 09:38:51,01/11/2019 09:39:20,"{'INTERSECTION': {'ORIGIN': {'ID': '', 'STREET...",01/11/2019 09:39:20,[{'value': 'Between E St and F St - Constructi...,"{'ROAD_CLOSED': False, 'EVENT': {'EVENT_ITEM_C...",ACTIVE,CONSTRUCTION,True


In [66]:
loc_all = pd.DataFrame(list(incident_df_2['location']))
loc_all

,GEOLOC,INTERSECTION,LENGTH,NAVTECH,POLITICAL_BOUNDARY,TPEGOpenLRBase64
0,"{'ORIGIN': {'LATITUDE': 42.3652, 'LONGITUDE': ...","{'ORIGIN': {'ID': '', 'STREET1': {'ADDRESS1': ...",0.10796,"{'EDGE': {'EDGE_ID': ['22013455']}, 'VERSION_I...","{'METRO_AREA': {'value': '', 'ID': 18}, 'COUNT...",NaN
1,"{'ORIGIN': {'LATITUDE': 42.33651, 'LONGITUDE':...","{'ORIGIN': {'ID': '', 'STREET1': {'ADDRESS1': ...",0.10602,"{'EDGE': {'EDGE_ID': ['22013004']}, 'VERSION_I...","{'METRO_AREA': {'value': '', 'ID': 18}, 'COUNT...",CCkBEAAlJM15lh4bIAAJBQQEA10ACgUEBIErAACc/5sACQ...


In [67]:
loc_intersec = pd.DataFrame(list(loc_all['INTERSECTION']))
loc_intersec

,ORIGIN,TO
0,"{'ID': '', 'STREET1': {'ADDRESS1': 'Friend St'...","{'ID': '', 'STREET1': {'ADDRESS1': 'Friend St'..."
1,"{'ID': '', 'STREET1': {'ADDRESS1': '5th St'}, ...","{'ID': '', 'STREET1': {'ADDRESS1': '5th St'}, ..."


In [68]:
loc_coord = pd.DataFrame(list(loc_all['GEOLOC']))
loc_coord

,GEOMETRY,ORIGIN,TO
0,"{'SHAPES': {'SHP': [{'value': '42.3652,-71.061...","{'LATITUDE': 42.3652, 'LONGITUDE': -71.06181}","[{'LATITUDE': 42.36398, 'LONGITUDE': -71.06049}]"
1,"{'SHAPES': {'SHP': [{'value': '42.33651,-71.05...","{'LATITUDE': 42.33651, 'LONGITUDE': -71.05085}","[{'LATITUDE': 42.3355, 'LONGITUDE': -71.04929}]"


In [77]:
loc_coord_shape = pd.Series(list(loc_coord['GEOMETRY']))
loc_coord_shape[0]

{'SHAPES': {'SHP': [{'value': '42.3652,-71.06181 42.36398,-71.06049',
    'FC': 5,
    'LID': '22013455T',
    'LE': 173.749167315973,
    'FW': 'SD'}]}}